## Data Acquisition & Preparation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset
data = pd.read_excel("online_retail_II.xlsx")

In [3]:
data.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


## Data Cleaning & Feature Engineering

In [5]:
# Remove cancellations and clean data
data = data[data['Quantity'] > 0]
data = data[data['Price'] > 0]
data['TotalAmount'] = data['Quantity'] * data['Price']
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Create reference date for recency calculation
reference_date = data['InvoiceDate'].max() + timedelta(days=1)

# Create customer-level features
customer_data = data.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalAmount': 'sum',
    'StockCode': lambda x: x.nunique(),
    'Country': 'first'
}).rename(columns={
        'StockCode': 'ProductVariety'
})

# Remove outliers
#Q1 = customer_data['TotalAmount'].quantile(0.25)
#Q3 = customer_data['TotalAmount'].quantile(0.75)
#IQR = Q3 - Q1
#customer_data = customer_data[
    #(customer_data['TotalAmount'] >= Q1 - 1.5*IQR) & 
    #(customer_data['TotalAmount'] <= Q3 + 1.5*IQR)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 511566 entries, 0 to 525460
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      511566 non-null  object        
 1   StockCode    511566 non-null  object        
 2   Description  511566 non-null  object        
 3   Quantity     511566 non-null  int64         
 4   InvoiceDate  511566 non-null  datetime64[ns]
 5   Price        511566 non-null  float64       
 6   Customer ID  407664 non-null  float64       
 7   Country      511566 non-null  object        
 8   TotalAmount  511566 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 39.0+ MB


## CUSTOMER SEGMENTATION

### Top 5% Most Valuable Customers

In [7]:
# Identify top 5% customers by revenue
customer_data['RevenuePercent'] = customer_data['TotalAmount'].rank(pct=True)
top_5_customers = customer_data[customer_data['RevenuePercent'] >= 0.95]
other_customers = customer_data[customer_data['RevenuePercent'] < 0.95]

# Calculate cumulative revenue
customer_data_sorted = customer_data.sort_values('TotalAmount', ascending=False)
customer_data_sorted['CumulativeRevenue'] = customer_data_sorted['TotalAmount'].cumsum()
customer_data_sorted['CumulativePercent'] = 100 * customer_data_sorted['CumulativeRevenue'] / customer_data_sorted['TotalAmount'].sum()

print(f"Top 5% customers: {len(top_5_customers)} customers")
print(f"Other_customers: {len(other_customers)} customers")

Top 5% customers: 216 customers
Other_customers: 4096 customers


In [8]:
# Analyze behavioral differences
behavior_comparison = pd.DataFrame({
    'Metric': ['Avg Order Value', 'Purchase Frequency', 'Product Variety', 'InvoiceDate'],
    'Top 5%': [
        other_customers['TotalAmount'].mean() / other_customers['Invoice'].mean(),
        other_customers['Invoice'].mean(),
        other_customers['ProductVariety'].mean(),
        other_customers['InvoiceDate'].mean()
    ],
    'All Customers': [
        customer_data['TotalAmount'].mean() / customer_data['Invoice'].mean(),
        customer_data['Invoice'].mean(),
        customer_data['ProductVariety'].mean(),
        customer_data['InvoiceDate'].mean()
    ]
})

In [9]:
# Analyze behavioral differences
behavior_comparison = pd.DataFrame({
    'Metric': ['Avg Order Value', 'Purchase Frequency', 'Product Variety', 'InvoiceDate'],
    'Top 5%': [
        top_5_customers['TotalAmount'].mean() / top_5_customers['Invoice'].mean(),
        top_5_customers['Invoice'].mean(),
        top_5_customers['ProductVariety'].mean(),
        top_5_customers['InvoiceDate'].mean()
    ],
    'All Customers': [
        customer_data['TotalAmount'].mean() / customer_data['Invoice'].mean(),
        customer_data['Invoice'].mean(),
        customer_data['ProductVariety'].mean(),
        customer_data['InvoiceDate'].mean()
    ]
})

In [10]:
customer_data['AOV'] = customer_data['TotalAmount'] / customer_data['Invoice']

# Identify top 5% customers by revenue (Monetary)
top_5_threshold = customer_data['TotalAmount'].quantile(0.95)
top_5_customers = customer_data[customer_data['TotalAmount'] >= top_5_threshold]
other_customers = customer_data[customer_data['TotalAmount'] < top_5_threshold]

print("\n=== AVERAGE ORDER VALUE CALCULATION ===")
print(f"Number of top 5% customers: {len(top_5_customers)}")
print(f"Number of other customers: {len(other_customers)}")

# Calculate mean AOV for both groups
top_5_aov = top_5_customers['AOV'].mean()
other_aov = other_customers['AOV'].mean()
aov_ratio = top_5_aov / other_aov

print(f"\nTop 5% customers AOV: ${top_5_aov:,.2f}")
print(f"Other customers AOV: ${other_aov:,.2f}")
print(f"Top 5% customers have {aov_ratio:.1f}x higher AOV")
print("-" * 50)


=== AVERAGE ORDER VALUE CALCULATION ===
Number of top 5% customers: 216
Number of other customers: 4096

Top 5% customers AOV: $1,207.20
Other customers AOV: $334.61
Top 5% customers have 3.6x higher AOV
--------------------------------------------------


In [11]:
# Frequency calculation
print("\n=== PURCHASE FREQUENCY CALCULATION ===")

# Calculate average frequency for both groups
top_5_freq = top_5_customers['Invoice'].mean()
other_freq = other_customers['Invoice'].mean()
freq_ratio = top_5_freq / other_freq

print(f"Top 5% customers average frequency: {top_5_freq:.1f} purchases")
print(f"Other customers average frequency: {other_freq:.1f} purchases")
print(f"Top 5% customers purchase {freq_ratio:.1f}x more frequently")
print("-" * 50)

# Let's look at distribution
print("\nFrequency distribution:")
print("Top 5% customers:")
print(f"  Min: {top_5_customers['Invoice'].min()}")
print(f"  Max: {top_5_customers['Invoice'].max()}")
print(f"  25th percentile: {top_5_customers['Invoice'].quantile(0.25):.0f}")
print(f"  50th percentile (median): {top_5_customers['Invoice'].median():.0f}")
print(f"  75th percentile: {top_5_customers['Invoice'].quantile(0.75):.0f}")

print("\nOther customers:")
print(f"  Min: {other_customers['Invoice'].min()}")
print(f"  Max: {other_customers['Invoice'].max()}")
print(f"  25th percentile: {other_customers['Invoice'].quantile(0.25):.0f}")
print(f"  50th percentile (median): {other_customers['Invoice'].median():.0f}")
print(f"  75th percentile: {other_customers['Invoice'].quantile(0.75):.0f}")

# Calculate the actual purchase intervals (time between purchases)
def calculate_purchase_intervals(customer_group):
    """Calculate average days between purchases for customer group"""
    intervals = []
    for customer_id in customer_group.index[:100]:  # Sample for efficiency
        customer_orders = data[data['Customer ID'] == customer_id]
        if len(customer_orders) > 1:
            order_dates = customer_orders['InvoiceDate'].sort_values()
            days_between = order_dates.diff().dt.days.mean()
            intervals.append(days_between)
    return np.mean(intervals) if intervals else np.nan

# Note: This calculation is computationally intensive for large datasets
# We'll use frequency as proxy for this demo


=== PURCHASE FREQUENCY CALCULATION ===
Top 5% customers average frequency: 23.7 purchases
Other customers average frequency: 3.4 purchases
Top 5% customers purchase 6.9x more frequently
--------------------------------------------------

Frequency distribution:
Top 5% customers:
  Min: 1
  Max: 205
  25th percentile: 9
  50th percentile (median): 16
  75th percentile: 26

Other customers:
  Min: 1
  Max: 62
  25th percentile: 1
  50th percentile (median): 2
  75th percentile: 4


In [12]:
print("\n=== PRODUCT VARIETY CALCULATION ===")

# Calculate average product variety
top_5_variety = top_5_customers['ProductVariety'].mean()
other_variety = other_customers['ProductVariety'].mean()
variety_ratio = top_5_variety / other_variety

print(f"Top 5% customers average product variety: {top_5_variety:.1f} unique products")
print(f"Other customers average product variety: {other_variety:.1f} unique products")
print(f"Top 5% customers buy {variety_ratio:.1f}x more product varieties")
print("-" * 50)

# Let's also look at product variety per purchase
top_5_variety_per_purchase = top_5_variety / top_5_freq
other_variety_per_purchase = other_variety / other_freq

print(f"\nProduct variety per purchase:")
print(f"Top 5%: {top_5_variety_per_purchase:.2f} unique products per purchase")
print(f"Other: {other_variety_per_purchase:.2f} unique products per purchase")
print(f"Top 5% try {top_5_variety_per_purchase/other_variety_per_purchase:.1f}x more new products per purchase")

# Calculate product category diversity
# First, we need to map products to categories (simplified example)
# In real analysis, you'd have product category mapping
print("\n=== PRODUCT CATEGORY ANALYSIS ===")

# Sample analysis of product diversity
sample_top_customer = top_5_customers.index[0]
sample_other_customer = other_customers.index[0]

top_customer_products = data[data['Customer ID'] == sample_top_customer]['Description'].unique()
other_customer_products = data[data['Customer ID'] == sample_other_customer]['Description'].unique()

print(f"Sample top customer ({sample_top_customer}):")
print(f"  Buys {len(top_customer_products)} different products")
print(f"  Sample products: {', '.join(top_customer_products[:3])}...")

print(f"\nSample other customer ({sample_other_customer}):")
print(f"  Buys {len(other_customer_products)} different products")
if len(other_customer_products) > 0:
    print(f"  Sample products: {', '.join(other_customer_products[:min(3, len(other_customer_products))])}...")


=== PRODUCT VARIETY CALCULATION ===
Top 5% customers average product variety: 208.7 unique products
Other customers average product variety: 56.0 unique products
Top 5% customers buy 3.7x more product varieties
--------------------------------------------------

Product variety per purchase:
Top 5%: 8.81 unique products per purchase
Other: 16.27 unique products per purchase
Top 5% try 0.5x more new products per purchase

=== PRODUCT CATEGORY ANALYSIS ===
Sample top customer (12357.0):
  Buys 165 different products
  Sample products: DOORMAT RED RETROSPOT, DOORMAT SPOTTY HOME SWEET HOME, DOORMAT FANCY FONT HOME SWEET HOME...

Sample other customer (12346.0):
  Buys 25 different products
  Sample products: This is a test product., RED SPOTTY CHILDS UMBRELLA, EDWARDIAN PARASOL RED...


In [13]:
print("\n" + "="*60)
print("COMPLETE SUMMARY: TOP 5% CUSTOMERS VS AVERAGE CUSTOMERS")
print("="*60)

summary_df = pd.DataFrame({
    'Metric': ['Average Order Value', 'Purchase Frequency', 
               'Product Variety','Total Revenue'],
    'Top 5% Customers': [
        f"${top_5_aov:,.0f}",
        f"{top_5_freq:.1f} purchases",
        f"{top_5_variety:.0f} unique products",
        f"${top_5_customers['TotalAmount'].sum():,.0f}",        
    ],
    'Other Customers': [
        f"${other_aov:,.0f}",
        f"{other_freq:.1f} purchases",
        f"{other_variety:.0f} unique products",
        f"${other_customers['TotalAmount'].sum():,.0f}",        
    ],
    'Ratio (Top 5% / Other)': [
        f"{aov_ratio:.1f}x",
        f"{freq_ratio:.1f}x",
        f"{variety_ratio:.1f}x",
        f"{(top_5_customers['TotalAmount'].sum()/other_customers['TotalAmount'].sum()):.1f}x more revenue",        
    ]
})

print(summary_df.to_string(index=False))
print("\n" + "="*60)
print("KEY BUSINESS INSIGHTS:")
print("="*60)
print("1. Top 5% customers have 3.6x higher Average Order Value")
print("   → They spend more per transaction")
print("   → Opportunity: Upsell premium products to these customers")
print()
print("2. They purchase 7x more frequently")
print("   → Higher engagement and loyalty")
print("   → Opportunity: Create subscription models or regular purchase incentives")
print()
print("3. They buy 3.7x more product varieties")
print("   → More exploratory buying behavior")
print("   → Opportunity: Cross-sell complementary products")
print()



COMPLETE SUMMARY: TOP 5% CUSTOMERS VS AVERAGE CUSTOMERS
             Metric    Top 5% Customers    Other Customers Ratio (Top 5% / Other)
Average Order Value              $1,207               $335                   3.6x
 Purchase Frequency      23.7 purchases      3.4 purchases                   6.9x
    Product Variety 209 unique products 56 unique products                   3.7x
      Total Revenue          $4,266,728         $4,565,275      0.9x more revenue

KEY BUSINESS INSIGHTS:
1. Top 5% customers have 3.6x higher Average Order Value
   → They spend more per transaction
   → Opportunity: Upsell premium products to these customers

2. They purchase 7x more frequently
   → Higher engagement and loyalty
   → Opportunity: Create subscription models or regular purchase incentives

3. They buy 3.7x more product varieties
   → More exploratory buying behavior
   → Opportunity: Cross-sell complementary products



### One-time vs Repeat Buyers Analysis

In [14]:
# One-time vs Repeat Buyers Analysis
# Identify one-time buyers
customer_purchase_count = data.groupby('Customer ID')['Invoice'].nunique()
one_time_buyers = customer_purchase_count[customer_purchase_count == 1].index
repeat_buyers = customer_purchase_count[customer_purchase_count > 1].index

# Analyze behavioral patterns
one_time_data = data[data['Customer ID'].isin(one_time_buyers)]
repeat_data = data[data['Customer ID'].isin(repeat_buyers)]

patterns_comparison = {
    'One-time Buyers': {
        'Avg Basket Size': one_time_data.groupby('Invoice')['Quantity'].sum().mean(),
        'Avg Order Value': one_time_data.groupby('Invoice')['TotalAmount'].sum().mean(),
        'Popular Categories': one_time_data.groupby('Description').size().nlargest(5).index.tolist(),
        'Return Rate': 0  # By definition
    },
    'Repeat Buyers': {
        'Avg Basket Size': repeat_data.groupby('Invoice')['Quantity'].sum().mean(),
        'Avg Order Value': repeat_data.groupby('Invoice')['TotalAmount'].sum().mean(),
        'Popular Categories': repeat_data.groupby('Description').size().nlargest(5).index.tolist(),
        'Avg Purchase Interval': repeat_data.groupby('Customer ID')['InvoiceDate'].apply(
            lambda x: x.diff().dt.days.mean()
        ).mean()
    }
}

### Customer Segmentation Strategy

In [15]:
def hybrid_segmentation(customer_df, transaction_df):
    """
    Create multi-dimensional segments combining:
    1. RFM (value & engagement)
    2. Product preferences (what they like)
    3. Geography (where they are)
    """
    
    # Step 1: Apply RFM segmentation
    print("Step 1: Applying RFM segmentation...")
    
    # First, let's define rfm_segmentation inside this function to avoid scope issues
    def rfm_segmentation(df):
        # Calculate R, F, M scores
        try:
            df['R_Score'] = pd.qcut(df['InvoiceDate'], 5, labels=[5,4,3,2,1], duplicates='drop')
        except:
            df['R_Score'] = pd.qcut(df['InvoiceDate'].rank(method='first'), 5, labels=[5,4,3,2,1])
        
        try:
            df['F_Score'] = pd.qcut(df['Invoice'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
        except:
            df['F_Score'] = pd.qcut(df['Invoice'], 5, labels=[1,2,3,4,5])
        
        try:
            df['M_Score'] = pd.qcut(df['TotalAmount'], 5, labels=[1,2,3,4,5], duplicates='drop')
        except:
            df['M_Score'] = pd.qcut(df['TotalAmount'].rank(method='first'), 5, labels=[1,2,3,4,5])
        
        # Convert to numeric
        df['R_Score'] = pd.to_numeric(df['R_Score'], errors='coerce')
        df['F_Score'] = pd.to_numeric(df['F_Score'], errors='coerce')
        df['M_Score'] = pd.to_numeric(df['M_Score'], errors='coerce')
        
        # Create RFM score string
        df['RFM_Score'] = df['R_Score'].astype(str) + df['F_Score'].astype(str) + df['M_Score'].astype(str)
        
        # Define segments
        conditions = [
            (df['R_Score'] >= 4) & (df['F_Score'] >= 4) & (df['M_Score'] >= 4),
            (df['R_Score'] >= 3) & (df['F_Score'] >= 3),
            (df['R_Score'] >= 3) & (df['F_Score'] >= 2),
            (df['R_Score'] <= 2) & (df['F_Score'] >= 2) & (df['M_Score'] >= 2),
            (df['R_Score'] <= 2) & (df['F_Score'] <= 2),
            (df['R_Score'] <= 1) & (df['F_Score'] <= 1)
        ]
        
        choices = ['Champions', 'Loyal Customers', 'Potential Loyalists', 
                   'At Risk', 'Hibernating', 'Lost']
        
        df['Segment'] = np.select(conditions, choices, default='Other')
        
        return df
    
    # Apply RFM segmentation
    customer_df = rfm_segmentation(customer_df)
    
    print("Step 2: Adding product preferences...")
    
    # Add product categories to transaction data
    def infer_category(description):
        if pd.isna(description):
            return 'Other'
        desc_lower = str(description).lower()
        
        # Simple category mapping
        if any(word in desc_lower for word in ['bag', 'case', 'holder', 'wallet']):
            return 'Accessories'
        elif any(word in desc_lower for word in ['light', 'lamp', 'candle', 'lantern']):
            return 'Lighting'
        elif any(word in desc_lower for word in ['ornament', 'decoration', 'garland']):
            return 'Decorations'
        elif any(word in desc_lower for word in ['pen', 'notebook', 'paper', 'card']):
            return 'Stationery'
        elif any(word in desc_lower for word in ['kitchen', 'plate', 'cup', 'bowl']):
            return 'Kitchenware'
        elif any(word in desc_lower for word in ['toy', 'game', 'puzzle']):
            return 'Toys'
        else:
            return 'Other'
    
    transaction_df['ProductCategory'] = transaction_df['Description'].apply(infer_category)
    
    # Get each customer's top product category
    customer_categories = transaction_df.groupby(['Customer ID', 'ProductCategory']).agg({
        'Quantity': 'sum'
    }).reset_index()
    
    customer_categories = customer_categories.sort_values(
        ['Customer ID', 'Quantity'], ascending=[True, False]
    )
    
    # Get primary category for each customer
    primary_category = customer_categories.drop_duplicates('Customer ID', keep='first')
    primary_category = primary_category[['Customer ID', 'ProductCategory']]
    primary_category.columns = ['Customer ID', 'Primary_Category']
    
    # Merge with customer data
    customer_df_reset = customer_df.reset_index()
    customer_df_reset = customer_df_reset.merge(primary_category, on='Customer ID', how='left')
    customer_df_reset['Primary_Category'] = customer_df_reset['Primary_Category'].fillna('Unknown')
    
    # Step 3: Add geographic segmentation
    print("Step 3: Adding geographic segmentation...")
    
    def get_geo_tier(country):
        if pd.isna(country):
            return 'Tier4'
        country_str = str(country)
        if country_str == 'United Kingdom':
            return 'Tier1'
        elif country_str in ['Germany', 'France', 'USA']:
            return 'Tier2'
        elif country_str in ['Spain', 'Italy', 'Netherlands']:
            return 'Tier3'
        else:
            return 'Tier4'
    
    customer_df_reset['Geo_Tier'] = customer_df_reset['Country'].apply(get_geo_tier)
    
    # Step 4: Create hybrid segment labels
    print("Step 4: Creating hybrid segment labels...")
    customer_df_reset['Hybrid_Segment'] = (
        customer_df_reset['Segment'] + '_' + 
        customer_df_reset['Primary_Category'] + '_' + 
        customer_df_reset['Geo_Tier']
    )
    
    # Step 5: Define campaign strategies
    print("Step 5: Defining campaign strategies...")
    
    segment_strategies = {
        'Champions': {
            'Campaign_Type': 'VIP Exclusive Offers',
            'Recommended_Discount': '15-20%',
            'Primary_Channel': 'Personalized Email + Phone'
        },
        'Loyal Customers': {
            'Campaign_Type': 'Loyalty Rewards',
            'Recommended_Discount': '10-15%',
            'Primary_Channel': 'Targeted Email'
        },
        'Potential Loyalists': {
            'Campaign_Type': 'Conversion Nurture',
            'Recommended_Discount': '10% first purchase',
            'Primary_Channel': 'Email + Retargeting Ads'
        },
        'At Risk': {
            'Campaign_Type': 'Win-Back Special',
            'Recommended_Discount': '20-25%',
            'Primary_Channel': 'Urgent Email + SMS'
        },
        'Hibernating': {
            'Campaign_Type': 'We Miss You',
            'Recommended_Discount': '25% comeback offer',
            'Primary_Channel': 'Email Only'
        },
        'Lost': {
            'Campaign_Type': 'Reactivation',
            'Recommended_Discount': '30%',
            'Primary_Channel': 'Bulk Email (low priority)'
        },
        'Other': {
            'Campaign_Type': 'General Newsletter',
            'Recommended_Discount': '5-10%',
            'Primary_Channel': 'Email'
        }
    }
    
    # Apply strategies
    for segment, strategy in segment_strategies.items():
        mask = customer_df_reset['Segment'] == segment
        for key, value in strategy.items():
            customer_df_reset.loc[mask, key] = value
    
    print("Hybrid segmentation completed successfully!")
    
    # Set index back to Customer ID
    customer_df_reset = customer_df_reset.set_index('Customer ID')
    
    return customer_df_reset

# Now you can run the function
print("Applying hybrid segmentation...")
customer_data_hybrid = hybrid_segmentation(customer_data.copy(), data.copy())
print(f"Segmentation complete! Created {len(customer_data_hybrid)} segments.")

Applying hybrid segmentation...
Step 1: Applying RFM segmentation...
Step 2: Adding product preferences...
Step 3: Adding geographic segmentation...
Step 4: Creating hybrid segment labels...
Step 5: Defining campaign strategies...
Hybrid segmentation completed successfully!
Segmentation complete! Created 4312 segments.


## CHURN PREDICTION

### Early Warning Signs

In [16]:
# Simple churn dataset
max_date = data['InvoiceDate'].max()
split_date = max_date - pd.Timedelta(days=90)
train_start = split_date - pd.Timedelta(days=365)

# Training data
train_data = data[(data['InvoiceDate'] >= train_start) & 
                  (data['InvoiceDate'] < split_date)]
future_data = data[data['InvoiceDate'] >= split_date]

# Create features
features = []
for cust_id in train_data['Customer ID'].unique():
    cust = train_data[train_data['Customer ID'] == cust_id]
    if len(cust) > 0:
        cust = cust.sort_values('InvoiceDate')
        recency = (split_date - cust['InvoiceDate'].max()).days
        frequency = cust['Invoice'].nunique()
        monetary = cust['TotalAmount'].sum()
        
        churn = 1 if cust_id not in future_data['Customer ID'].unique() else 0
        
        features.append([recency, frequency, monetary, churn])

# Convert to DataFrame
df_features = pd.DataFrame(features, columns=['Recency', 'Frequency', 'Monetary', 'Churn'])

# Train model
X = df_features[['Recency', 'Frequency', 'Monetary']]
y = df_features['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Results
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2%}")

# Feature importance
for feat, imp in zip(['Recency', 'Frequency', 'Monetary'], model.feature_importances_):
    print(f"{feat}: {imp:.1%}")

print(f"\nChurn rate: {y.mean():.1%}")
print(f"Customers: {len(df_features):,}")

Accuracy: 61.83%
Recency: 37.4%
Frequency: 10.9%
Monetary: 51.6%

Churn rate: 42.5%
Customers: 3,377


### Cost-Effective Interventions

In [17]:
# FIRST: We need to create segments for churn analysis

def create_segments_for_churn(customer_data):
    # Make a copy to avoid modifying original
    df = customer_data.copy()
    
    # Calculate RFM scores (1-5 scale)
    df['R_Score'] = pd.qcut(df['InvoiceDate'].rank(method='first'), 
                            5, labels=[5,4,3,2,1])
    df['F_Score'] = pd.qcut(df['Invoice'].rank(method='first'), 
                            5, labels=[1,2,3,4,5])
    df['M_Score'] = pd.qcut(df['TotalAmount'].rank(method='first'), 
                            5, labels=[1,2,3,4,5])
    
    # Convert to numeric
    df['R_Score'] = pd.to_numeric(df['R_Score'])
    df['F_Score'] = pd.to_numeric(df['F_Score'])
    df['M_Score'] = pd.to_numeric(df['M_Score'])
    
    # Create segments based on RFM scores
    conditions = [
        (df['R_Score'] >= 4) & (df['F_Score'] >= 4) & (df['M_Score'] >= 4),
        (df['R_Score'] >= 3) & (df['F_Score'] >= 3),
        (df['R_Score'] >= 3) & (df['F_Score'] >= 2),
        (df['R_Score'] <= 2) & (df['F_Score'] >= 2) & (df['M_Score'] >= 2),
        (df['R_Score'] <= 2) & (df['F_Score'] <= 2),
        (df['R_Score'] <= 1) & (df['F_Score'] <= 1)
    ]
    
    choices = ['Champions', 'Loyal', 'Potential', 'At Risk', 'Hibernating', 'Lost']
    
    df['Segment'] = np.select(conditions, choices, default='Other')
    
    return df

# Apply segmentation to customer_data
customer_data = create_segments_for_churn(customer_data)

print("\nSegment Distribution for Churn Analysis:")
print(customer_data['Segment'].value_counts())


def calculate_clv_for_segments(df, monthly_discount_rate=0.01):
    """
    Calculate Customer Lifetime Value for each segment
    """
    # Simple CLV calculation
    # Avg monthly purchase * Avg customer lifespan (months)
    avg_monthly_purchase = df['TotalAmount'] / (df['Invoice']/30).clip(lower=1)
    avg_lifespan = 12  # Based on data analysis
    df['CLV'] = avg_monthly_purchase * avg_lifespan / (1 + monthly_discount_rate)
    return df

# Calculate CLV
customer_data = calculate_clv_for_segments(customer_data)

print(f"\nCLV by Segment:")
print(customer_data.groupby('Segment')['CLV'].mean().round(2).sort_values(ascending=False))

# ============================================
# Cost-Effective Interventions by Segment
# ============================================

def analyze_interventions_by_segment(customer_data):
    """
    Analyze which interventions are most cost-effective for each segment
    """
    
    # Define interventions with cost and effectiveness
    interventions = {
        'Personalized_Email': {
            'cost_per_customer': 2.00,
            'effectiveness': 0.15,  # 15% reduction in churn probability
            'description': 'Personalized re-engagement email'
        },
        'Discount_10%': {
            'cost_per_customer': 0.10,  # 10% of average order value
            'effectiveness': 0.25,
            'description': '10% discount on next purchase'
        },
        'Free_Shipping': {
            'cost_per_customer': 5.00,
            'effectiveness': 0.20,
            'description': 'Free shipping offer'
        },
        'Product_Recommendations': {
            'cost_per_customer': 3.00,
            'effectiveness': 0.18,
            'description': 'Personalized product recommendations'
        },
        'Loyalty_Points': {
            'cost_per_customer': 1.50,
            'effectiveness': 0.12,
            'description': 'Bonus loyalty points'
        }
    }
    
    # Calculate intervention ROI by segment
    roi_results = {}
    
    for segment in customer_data['Segment'].unique():
        segment_data = customer_data[customer_data['Segment'] == segment]
        
        if len(segment_data) == 0:
            continue
        
        # Calculate segment metrics
        avg_clv = segment_data['CLV'].mean()
        segment_size = len(segment_data)
        
        segment_roi = {}
        
        for intervention_name, intervention in interventions.items():
            cost = intervention['cost_per_customer']
            effectiveness = intervention['effectiveness']
            
            # Expected value saved per customer
            expected_savings = avg_clv * effectiveness
            
            # ROI calculation
            roi = (expected_savings - cost) / cost if cost > 0 else float('inf')
            
            # Expected savings for entire segment
            total_savings = expected_savings * segment_size
            total_cost = cost * segment_size
            
            segment_roi[intervention_name] = {
                'roi': roi,
                'expected_savings_per_customer': expected_savings,
                'total_savings': total_savings,
                'total_cost': total_cost,
                'description': intervention['description']
            }
        
        roi_results[segment] = segment_roi
    
    return roi_results

# Analyze interventions
roi_analysis = analyze_interventions_by_segment(customer_data)

print("\n" + "="*70)
print("COST-EFFECTIVE INTERVENTIONS BY SEGMENT")
print("="*70)

for segment, interventions in roi_analysis.items():
    print(f"\n{segment} Segment:")
    print("-" * 40)
    
    # Sort interventions by ROI
    sorted_interventions = sorted(interventions.items(), 
                                  key=lambda x: x[1]['roi'], 
                                  reverse=True)
    
    for i, (intervention_name, data) in enumerate(sorted_interventions[:3], 1):
        print(f"{i}. {intervention_name}:")
        print(f"   ROI: {data['roi']:.1f}x")
        print(f"   Cost per customer: ${data['total_cost']/len(customer_data[customer_data['Segment']==segment]):.2f}")
        print(f"   Expected savings: ${data['expected_savings_per_customer']:.2f}")
        print(f"   Description: {data['description']}")

# ============================================
# Create Recommended Actions
# ============================================

def create_segment_intervention_recommendations(roi_analysis):
    """
    Create specific recommendations for each segment
    """
    recommendations = {}
    
    for segment, interventions in roi_analysis.items():
        # Find best intervention (highest ROI)
        best_intervention = max(interventions.items(), 
                                key=lambda x: x[1]['roi'])
        
        intervention_name = best_intervention[0]
        intervention_data = best_intervention[1]
        
        # Segment-specific messaging
        messaging = {
            'Champions': f"Exclusive early access + {intervention_name}",
            'Loyal': f"Thank you offer + {intervention_name}",
            'Potential': f"Welcome back + {intervention_name}",
            'At Risk': f"We miss you! + {intervention_name}",
            'Hibernating': f"Come back special + {intervention_name}",
            'Lost': f"Final chance + {intervention_name}"
        }
        
        recommendations[segment] = {
            'recommended_intervention': intervention_name,
            'roi': intervention_data['roi'],
            'expected_savings': intervention_data['expected_savings_per_customer'],
            'message': messaging.get(segment, f"Special offer: {intervention_name}"),
            'priority': 'High' if segment in ['At Risk', 'Hibernating'] else 'Medium'
        }
    
    return recommendations

# Create recommendations
recommendations = create_segment_intervention_recommendations(roi_analysis)

print("\n" + "="*70)
print("RECOMMENDED INTERVENTIONS")
print("="*70)

for segment, rec in recommendations.items():
    print(f"\n{segment}:")
    print(f"  Intervention: {rec['recommended_intervention']}")
    print(f"  Message: {rec['message']}")
    print(f"  Expected ROI: {rec['roi']:.1f}x")
    print(f"  Priority: {rec['priority']}")

# ============================================
# Budget Allocation Recommendations
# ============================================

def calculate_budget_allocation(customer_data, recommendations):
    """
    Calculate optimal budget allocation across segments
    """
    budget_summary = []
    
    for segment, rec in recommendations.items():
        segment_data = customer_data[customer_data['Segment'] == segment]
        
        if len(segment_data) == 0:
            continue
        
        segment_size = len(segment_data)
        avg_clv = segment_data['CLV'].mean()
        
        # Recommended budget: 30% of expected savings
        total_expected_savings = avg_clv * 0.25 * segment_size  # 25% retention rate
        recommended_budget = total_expected_savings * 0.3
        
        budget_summary.append({
            'Segment': segment,
            'Customers': segment_size,
            'Avg_CLV': avg_clv,
            'Total_At_Risk': segment_size * 0.6,  # Assume 60% at risk
            'Expected_Savings': total_expected_savings,
            'Recommended_Budget': recommended_budget,
            'ROI': rec['roi']
        })
    
    return pd.DataFrame(budget_summary)

# Calculate budget allocation
budget_df = calculate_budget_allocation(customer_data, recommendations)

print("\n" + "="*70)
print("BUDGET ALLOCATION RECOMMENDATIONS")
print("="*70)
print(budget_df.to_string(index=False))

# Total budget needed
total_budget = budget_df['Recommended_Budget'].sum()
print(f"\nTotal Recommended Budget: ${total_budget:,.2f}")
print(f"Total Expected Savings: ${budget_df['Expected_Savings'].sum():,.2f}")
print(f"Overall Expected ROI: {budget_df['Expected_Savings'].sum()/total_budget:.1f}x")

# Create final output DataFrame
output_data = customer_data.copy()

# Add intervention recommendations
segment_to_intervention = {seg: rec['recommended_intervention'] 
                          for seg, rec in recommendations.items()}
output_data['Recommended_Intervention'] = output_data['Segment'].map(segment_to_intervention)

# Add ROI information
segment_to_roi = {seg: rec['roi'] for seg, rec in recommendations.items()}
output_data['Expected_ROI'] = output_data['Segment'].map(segment_to_roi)

# Save for Power BI
output_data.to_csv('churn_intervention_recommendations.csv', index=True)
print(f"\nSaved recommendations to 'churn_intervention_recommendations.csv'")


Segment Distribution for Churn Analysis:
Segment
Loyal          976
Champions      911
At Risk        898
Hibernating    773
Potential      390
Other          364
Name: count, dtype: int64

CLV by Segment:
Segment
Champions      58331.52
Loyal          16469.67
At Risk        16275.76
Potential       6400.54
Other           4224.90
Hibernating     3644.94
Name: CLV, dtype: float64

COST-EFFECTIVE INTERVENTIONS BY SEGMENT

At Risk Segment:
----------------------------------------
1. Discount_10%:
   ROI: 40688.4x
   Cost per customer: $0.10
   Expected savings: $4068.94
   Description: 10% discount on next purchase
2. Loyalty_Points:
   ROI: 1301.1x
   Cost per customer: $1.50
   Expected savings: $1953.09
   Description: Bonus loyalty points
3. Personalized_Email:
   ROI: 1219.7x
   Cost per customer: $2.00
   Expected savings: $2441.36
   Description: Personalized re-engagement email

Potential Segment:
----------------------------------------
1. Discount_10%:
   ROI: 16000.4x
   Cos

In [18]:
def create_churn_labels(df, churn_threshold=90):
    """
    Create churn labels based on recency threshold
    """
    df = df.copy()
    
    # Define churn: No purchase in last 'churn_threshold' days
    df['Churn'] = (df['InvoiceDate'] > churn_threshold).astype(int)
    
    return df

# Apply churn labeling
customer_data = create_churn_labels(customer_data, churn_threshold=90)

print(f"\nChurn distribution:")
print(f"Active customers (Churn=0): {(customer_data['Churn'] == 0).sum()}")
print(f"Churned customers (Churn=1): {(customer_data['Churn'] == 1).sum()}")
print(f"Churn rate: {customer_data['Churn'].mean():.1%}")



Churn distribution:
Active customers (Churn=0): 2877
Churned customers (Churn=1): 1435
Churn rate: 33.3%


### Potential Loss & Budget Allocation

In [19]:
# ============================================
# Potential Loss & Budget Allocation
# ============================================

print("\n" + "="*70)
print("POTENTIAL LOSS & BUDGET ALLOCATION ANALYSIS")
print("="*70)

def calculate_potential_loss(customer_data, churn_threshold=90, retention_rate=0.25):
    """
    Calculate potential loss from at-risk customers
    """
    
    # Identify at-risk customers (currently defined as churned or about to churn)
    # Using current churners and customers with high recency
    high_risk_threshold = churn_threshold * 0.7  # 70% of churn threshold
    
    # At-risk customers: Already churned OR high recency
    at_risk_mask = (customer_data['Churn'] == 1) | (customer_data['InvoiceDate'] > high_risk_threshold)
    at_risk_customers = customer_data[at_risk_mask]
    
    # Calculate total CLV at risk
    total_clv_at_risk = at_risk_customers['CLV'].sum()
    
    # Calculate savable CLV with interventions
    # Assuming we can save retention_rate of at-risk customers
    savable_clv = total_clv_at_risk * retention_rate
    
    # Optimal budget allocation (spend 30% of savable value)
    optimal_budget = savable_clv * 0.3
    
    # Expected ROI
    expected_roi = (savable_clv - optimal_budget) / optimal_budget
    
    return {
        'at_risk_customers': at_risk_customers,
        'total_clv_at_risk': total_clv_at_risk,
        'savable_clv': savable_clv,
        'optimal_budget': optimal_budget,
        'expected_roi': expected_roi,
        'retention_rate': retention_rate
    }

# Calculate potential loss
results = calculate_potential_loss(customer_data)

print(f"\nAt-Risk Customer Analysis:")
print(f"Total customers at risk: {len(results['at_risk_customers'])}")
print(f"Percentage of total customers: {len(results['at_risk_customers'])/len(customer_data):.1%}")

print(f"\nCLV at Risk:")
print(f"Total CLV at risk: ${results['total_clv_at_risk']:,.0f}")
print(f"Average CLV per at-risk customer: ${results['total_clv_at_risk']/len(results['at_risk_customers']):,.0f}")

print(f"\nRetention Potential (assuming {results['retention_rate']:.0%} success rate):")
print(f"Savable CLV with interventions: ${results['savable_clv']:,.0f}")

print(f"\nBudget Allocation Recommendation:")
print(f"Optimal retention budget: ${results['optimal_budget']:,.0f}")
print(f"Expected ROI: {results['expected_roi']:.1f}x ({results['expected_roi']*100:.0f}%)")



POTENTIAL LOSS & BUDGET ALLOCATION ANALYSIS

At-Risk Customer Analysis:
Total customers at risk: 1910
Percentage of total customers: 44.3%

CLV at Risk:
Total CLV at risk: $19,833,043
Average CLV per at-risk customer: $10,384

Retention Potential (assuming 25% success rate):
Savable CLV with interventions: $4,958,261

Budget Allocation Recommendation:
Optimal retention budget: $1,487,478
Expected ROI: 2.3x (233%)


In [20]:
# ============================================
# Segment-Specific Analysis
# ============================================

print("\n" + "="*70)
print("SEGMENT-SPECIFIC RISK ANALYSIS")
print("="*70)

def analyze_segment_risk(customer_data):
    """
    Analyze risk and CLV loss by segment
    """
    segment_analysis = []
    
    for segment in customer_data['Segment'].unique():
        segment_data = customer_data[customer_data['Segment'] == segment]
        
        # Calculate segment metrics
        segment_size = len(segment_data)
        churn_rate = segment_data['Churn'].mean()
        total_clv = segment_data['CLV'].sum()
        avg_clv = segment_data['CLV'].mean()
        
        # At-risk customers in segment
        at_risk_in_segment = segment_data[segment_data['Churn'] == 1]
        at_risk_count = len(at_risk_in_segment)
        clv_at_risk = at_risk_in_segment['CLV'].sum()
        
        # Calculate priority
        if churn_rate > 0.5:
            priority = 'Critical'
        elif churn_rate > 0.3:
            priority = 'High'
        elif churn_rate > 0.1:
            priority = 'Medium'
        else:
            priority = 'Low'
        
        segment_analysis.append({
            'Segment': segment,
            'Customers': segment_size,
            'Churn_Rate': churn_rate,
            'At_Risk_Customers': at_risk_count,
            'CLV_At_Risk': clv_at_risk,
            'Avg_CLV': avg_clv,
            'Priority': priority
        })
    
    return pd.DataFrame(segment_analysis)

# Analyze by segment
segment_risk_df = analyze_segment_risk(customer_data)

print("\nRisk Analysis by Segment:")
print(segment_risk_df.sort_values('CLV_At_Risk', ascending=False).to_string(index=False))



SEGMENT-SPECIFIC RISK ANALYSIS

Risk Analysis by Segment:
    Segment  Customers  Churn_Rate  At_Risk_Customers  CLV_At_Risk      Avg_CLV Priority
    At Risk        898    0.781737                702 1.045495e+07 16275.760504 Critical
Hibernating        773    0.886158                685 2.276065e+06  3644.944767 Critical
      Other        364    0.131868                 48 9.410032e+04  4224.903819   Medium
  Potential        390    0.000000                  0 0.000000e+00  6400.543184      Low
      Loyal        976    0.000000                  0 0.000000e+00 16469.670920      Low
  Champions        911    0.000000                  0 0.000000e+00 58331.524610      Low


In [21]:
print("\n" + "="*70)
print("DETAILED BUDGET ALLOCATION BY SEGMENT")
print("="*70)

def allocate_budget_by_segment(segment_risk_df, total_budget):
    """
    Allocate budget based on segment risk and potential value
    """
    # Calculate allocation weights
    # Weight = (CLV_At_Risk * Churn_Rate) for each segment
    segment_risk_df['Allocation_Weight'] = segment_risk_df['CLV_At_Risk'] * segment_risk_df['Churn_Rate']
    total_weight = segment_risk_df['Allocation_Weight'].sum()
    
    # Allocate budget proportionally
    segment_risk_df['Budget_Allocation'] = (segment_risk_df['Allocation_Weight'] / total_weight) * total_budget
    
    # Calculate expected return
    # Assume 25% retention success rate
    retention_success_rate = 0.25
    segment_risk_df['Expected_Return'] = segment_risk_df['CLV_At_Risk'] * retention_success_rate
    segment_risk_df['Expected_ROI'] = (segment_risk_df['Expected_Return'] - segment_risk_df['Budget_Allocation']) / segment_risk_df['Budget_Allocation']
    
    # Recommended actions
    actions = {
        'Champions': 'Preventive retention (exclusive offers)',
        'Loyal Customers': 'Relationship building (personalized service)',
        'Potential Loyalists': 'Conversion nurturing (loyalty program)',
        'At Risk': 'Urgent win-back (significant discounts)',
        'Hibernating': 'Reactivation campaigns (comeback offers)',
        'Lost': 'Low-cost reactivation (email campaigns)',
        'Other': 'General retention (improved service)'
    }
    
    segment_risk_df['Recommended_Action'] = segment_risk_df['Segment'].map(actions)
    
    return segment_risk_df

# Allocate budget
total_budget = results['optimal_budget']  # Use optimal budget from previous calculation
budget_allocation_df = allocate_budget_by_segment(segment_risk_df, total_budget)

# Display allocation
print("\nRecommended Budget Allocation:")
allocation_display = budget_allocation_df[['Segment', 'Customers', 'Churn_Rate', 
                                          'CLV_At_Risk', 'Budget_Allocation', 
                                          'Expected_Return', 'Expected_ROI', 
                                          'Recommended_Action']].copy()

allocation_display['Budget_Allocation'] = allocation_display['Budget_Allocation'].apply(lambda x: f"${x:,.0f}")
allocation_display['Expected_Return'] = allocation_display['Expected_Return'].apply(lambda x: f"${x:,.0f}")
allocation_display['Expected_ROI'] = allocation_display['Expected_ROI'].apply(lambda x: f"{x:.1f}x")

print(allocation_display.to_string(index=False))

# Summary
total_allocated = budget_allocation_df['Budget_Allocation'].sum()
total_expected_return = budget_allocation_df['Expected_Return'].sum()
overall_roi = (total_expected_return - total_allocated) / total_allocated

print(f"\nSummary:")
print(f"Total Budget Allocated: ${total_allocated:,.0f}")
print(f"Total Expected Return: ${total_expected_return:,.0f}")
print(f"Overall Expected ROI: {overall_roi:.1f}x ({overall_roi*100:.0f}%)")



DETAILED BUDGET ALLOCATION BY SEGMENT

Recommended Budget Allocation:
    Segment  Customers  Churn_Rate  CLV_At_Risk Budget_Allocation Expected_Return Expected_ROI                       Recommended_Action
    At Risk        898    0.781737 1.045495e+07        $1,191,603      $2,613,738         1.2x  Urgent win-back (significant discounts)
  Potential        390    0.000000 0.000000e+00                $0              $0         nanx                                      NaN
Hibernating        773    0.886158 2.276065e+06          $294,066        $569,016         0.9x Reactivation campaigns (comeback offers)
      Loyal        976    0.000000 0.000000e+00                $0              $0         nanx                                      NaN
      Other        364    0.131868 9.410032e+04            $1,809         $23,525        12.0x     General retention (improved service)
  Champions        911    0.000000 0.000000e+00                $0              $0         nanx  Preventive retent

## STRATEGIC RECOMMENDATIONS

### Loyalty Program Design

In [22]:
# Realistic loyalty program analysis
def analyze_loyalty_program_realistic(customer_data):
    """
    Realistic loyalty program ROI analysis
    """
    
    print("="*70)
    print("REALISTIC LOYALTY PROGRAM FEASIBILITY ANALYSIS")
    print("="*70)
    
    # Calculate CLV properly (using industry standard)
    def calculate_realistic_clv(df):
        """Calculate realistic Customer Lifetime Value"""
        # Average monthly revenue per customer
        avg_monthly_revenue = df['TotalAmount'] / (df['InvoiceDate']/30).clip(lower=1)
        
        # Average customer lifespan (based on data)
        # Customers with recency > 180 days are considered lost
        active_customers = df[df['InvoiceDate'] <= 180]
        avg_lifespan_months = 12  # Conservative estimate
        
        # Gross margin (assume 40% for online retail)
        gross_margin = 0.40
        
        # Discount rate (8% annual)
        monthly_discount = 0.08/12
        
        # Calculate CLV
        clv = avg_monthly_revenue * gross_margin * avg_lifespan_months
        return clv
    
    customer_data['CLV'] = calculate_realistic_clv(customer_data)
    
    # Define realistic tiers
    tiers = {
        'Silver': {
            'Criteria': (customer_data['CLV'] > 100) & (customer_data['Invoice'] >= 2),
            'Benefits': ['Free shipping', 'Birthday gift'],
            'Annual_cost_per_customer': 25,  # Realistic cost
            'Expected_CLV_increase': 0.10,   # 10% increase
            'Retention_boost': 0.05,         # 5% retention increase
            'Customers': 0
        },
        'Gold': {
            'Criteria': (customer_data['CLV'] > 300) & (customer_data['Invoice'] >= 5),
            'Benefits': ['15% discounts', 'Priority support', 'Free returns'],
            'Annual_cost_per_customer': 75,
            'Expected_CLV_increase': 0.15,
            'Retention_boost': 0.08,
            'Customers': 0
        },
        'Platinum': {
            'Criteria': (customer_data['CLV'] > 600) & (customer_data['Invoice'] >= 10),
            'Benefits': ['25% discounts', 'Personal shopper', 'Exclusive events'],
            'Annual_cost_per_customer': 200,
            'Expected_CLV_increase': 0.20,
            'Retention_boost': 0.10,
            'Customers': 0
        }
    }
    
    # Count customers in each tier
    for tier, details in tiers.items():
        mask = details['Criteria']
        tiers[tier]['Customers'] = len(customer_data[mask])
        if tiers[tier]['Customers'] > 0:
            tiers[tier]['Avg_CLV'] = customer_data.loc[mask, 'CLV'].mean()
        else:
            tiers[tier]['Avg_CLV'] = 0
    
    print("\nTier Distribution:")
    for tier, details in tiers.items():
        print(f"  {tier}: {details['Customers']} customers, Avg CLV: ${details['Avg_CLV']:.0f}")
    
    # Realistic costs
    print("\nAnnual Program Costs:")
    total_annual_cost = 0
    
    for tier, details in tiers.items():
        tier_cost = details['Customers'] * details['Annual_cost_per_customer']
        total_annual_cost += tier_cost
        print(f"  {tier}: ${tier_cost:,.0f}")
    
    # One-time setup costs
    setup_costs = {
        'Technology_platform': 50000,
        'Marketing_launch': 30000,
        'Staff_training': 20000,
        'Legal_compliance': 15000
    }
    
    total_setup = sum(setup_costs.values())
    print(f"\nOne-time Setup Costs: ${total_setup:,.0f}")
    
    # Annual benefits
    print("\nAnnual Program Benefits:")
    total_annual_benefit = 0
    
    for tier, details in tiers.items():
        if details['Customers'] > 0:
            # CLV increase from higher spending
            clv_increase = details['Customers'] * details['Avg_CLV'] * details['Expected_CLV_increase']
            
            # Retention benefit (reducing churn)
            retention_benefit = details['Customers'] * details['Avg_CLV'] * details['Retention_boost']
            
            tier_benefit = clv_increase + retention_benefit
            total_annual_benefit += tier_benefit
            
            print(f"  {tier}:")
            print(f"    CLV increase: ${clv_increase:,.0f}")
            print(f"    Retention benefit: ${retention_benefit:,.0f}")
            print(f"    Total: ${tier_benefit:,.0f}")
    
    # ROI Calculation
    print("\n" + "="*70)
    print("ROI ANALYSIS (5-Year Horizon)")
    print("="*70)
    
    # 5-year projection
    years = 5
    total_costs = total_setup + (total_annual_cost * years)
    total_benefits = total_annual_benefit * years
    
    # Account for diminishing returns
    # Benefits may decrease 10% each year as novelty wears off
    adjusted_benefits = 0
    for year in range(1, years + 1):
        diminishing_factor = 0.9 ** (year - 1)  # 10% decrease each year
        adjusted_benefits += total_annual_benefit * diminishing_factor
    
    roi = (adjusted_benefits - total_costs) / total_costs
    payback_period = total_setup / (total_annual_benefit - total_annual_cost)
    
    print(f"Total 5-Year Costs: ${total_costs:,.0f}")
    print(f"Total 5-Year Benefits (adjusted): ${adjusted_benefits:,.0f}")
    print(f"ROI: {roi:.1%}")
    print(f"Payback Period: {payback_period:.1f} years")
    
    # Break-even analysis
    break_even_customers = total_setup / ((customer_data['CLV'].mean() * 0.15) - 50)
    print(f"\nBreak-even: Need {break_even_customers:.0f} active members")
    
    return tiers, roi, payback_period

# Run realistic analysis
tiers, roi, payback_period = analyze_loyalty_program_realistic(customer_data)

REALISTIC LOYALTY PROGRAM FEASIBILITY ANALYSIS

Tier Distribution:
  Silver: 2872 customers, Avg CLV: $11752
  Gold: 1177 customers, Avg CLV: $24406
  Platinum: 403 customers, Avg CLV: $52216

Annual Program Costs:
  Silver: $71,800
  Gold: $88,275
  Platinum: $80,600

One-time Setup Costs: $115,000

Annual Program Benefits:
  Silver:
    CLV increase: $3,375,222
    Retention benefit: $1,687,611
    Total: $5,062,833
  Gold:
    CLV increase: $4,308,863
    Retention benefit: $2,298,060
    Total: $6,606,924
  Platinum:
    CLV increase: $4,208,619
    Retention benefit: $2,104,309
    Total: $6,312,928

ROI ANALYSIS (5-Year Horizon)
Total 5-Year Costs: $1,318,375
Total 5-Year Benefits (adjusted): $73,640,892
ROI: 5485.7%
Payback Period: 0.0 years

Break-even: Need 99 active members


### Product Recommendations for At-Risk Customers

In [23]:
# Load the data
data = pd.read_excel("online_retail_II.xlsx")

# Remove cancellations and clean data
data = data[data['Quantity'] > 0]
data = data[data['Price'] > 0]
data['TotalAmount'] = data['Quantity'] * data['Price']
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Create reference date for recency calculation
reference_date = data['InvoiceDate'].max() + timedelta(days=1)

# Create customer-level features
customer_data = data.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalAmount': 'sum',
    'StockCode': lambda x: x.nunique(),
    'Country': 'first'
}).rename(columns={
        'StockCode': 'ProductVariety'
})


In [24]:
# MOST PRACTICAL VERSION

def create_product_recommendations_engine(data):
    """
    Create a simple but effective recommendation engine
    """
    
    # 1. Clean product descriptions
    data['Description'] = data['Description'].astype(str).str.strip()
    
    # 2. Find best sellers (global)
    best_sellers = data.groupby('Description').agg({
        'Quantity': 'sum',
        'Customer ID': 'nunique'
    }).sort_values('Quantity', ascending=False).head(20)
    
    # 3. Find trending products (last 30 days)
    recent_threshold = data['InvoiceDate'].max() - pd.Timedelta(days=30)
    recent_sales = data[data['InvoiceDate'] > recent_threshold]
    trending = recent_sales.groupby('Description')['Quantity'].sum().nlargest(10).index.tolist()
    
    # 4. Create customer purchase history
    customer_history = {}
    for cust_id in data['Customer ID'].unique():
        cust_data = data[data['Customer ID'] == cust_id]
        purchased = cust_data['Description'].unique()
        customer_history[cust_id] = set(purchased)
    
    # 5. Recommendation function
    def recommend_for_customer(customer_id, num_recommendations=3):
        if customer_id not in customer_history:
            # New customer - recommend best sellers
            return best_sellers.index[:num_recommendations].tolist()
        
        # What they've bought
        bought = customer_history[customer_id]
        
        # Strategy 1: Recommend trending products they haven't bought
        recommendations = []
        for product in trending:
            if product not in bought:
                recommendations.append(product)
                if len(recommendations) >= num_recommendations:
                    break
        
        # Strategy 2: If not enough trending, add best sellers
        if len(recommendations) < num_recommendations:
            for product in best_sellers.index:
                if product not in bought and product not in recommendations:
                    recommendations.append(product)
                    if len(recommendations) >= num_recommendations:
                        break
        
        return recommendations
    
    return recommend_for_customer

# Create recommendation engine
recommend_engine = create_product_recommendations_engine(data)

# Test with example at-risk customers
at_risk_examples = data['Customer ID'].dropna().unique()[:5]

print("Product Recommendations for At-Risk Customers:")
print("="*50)

for customer_id in at_risk_examples:
    recommendations = recommend_engine(customer_id)
    print(f"\nCustomer {int(customer_id)}:")
    for i, product in enumerate(recommendations, 1):
        print(f"  {i}. {product}")

Product Recommendations for At-Risk Customers:

Customer 13085:
  1. ASSORTED COLOUR BIRD ORNAMENT
  2. SMALL POPCORN HOLDER
  3. WHITE HANGING HEART T-LIGHT HOLDER

Customer 13078:
  1. SMALL POPCORN HOLDER
  2. WHITE HANGING HEART T-LIGHT HOLDER
  3. JUMBO BAG RED RETROSPOT

Customer 15362:
  1. ASSORTED COLOUR BIRD ORNAMENT
  2. SMALL POPCORN HOLDER
  3. WHITE HANGING HEART T-LIGHT HOLDER

Customer 18102:
  1. ASSORTED COLOUR BIRD ORNAMENT
  2. SMALL POPCORN HOLDER
  3. JUMBO BAG RED RETROSPOT

Customer 12682:
  1. ASSORTED COLOUR BIRD ORNAMENT
  2. WHITE HANGING HEART T-LIGHT HOLDER
  3. WORLD WAR 2 GLIDERS ASSTD DESIGNS
